In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from PIL import Image
import time

# Define constants for file paths
INPUT_FOLDER = r"C:\Users\abudh\Desktop\CropWatch\Test_15_08"
SPLICE_TEST_FOLDER = r"C:\Users\abudh\Desktop\CropWatch\spliced_image"
FILTERED_IMAGE_FOLDER = r'C:\Users\abudh\Desktop\CropWatch\Filtered_Image_Platform1'
TFLITE_MODEL_PATH = r'C:\Users\abudh\Desktop\CropWatch\16_08_FInal\best_model_Platform1A.tflite'


# Ensure the spliced image folder exists
os.makedirs(SPLICE_TEST_FOLDER, exist_ok=True)

def splice_images(input_folder, splice_test_folder):
    for filename in os.listdir(input_folder):
        img_path = os.path.join(input_folder, filename)
        try:
            with Image.open(img_path) as img:
                width, height = img.size
                if width != 540 or height != 405:
                    continue  # Skip non-standard images
                
                quadrants = [
                    (0, 0, width // 2, height // 2),
                    (width // 2, 0, width, height // 2),
                    (0, height // 2, width // 2, height),
                    (width // 2, height // 2, width, height)
                ]
                
                for i, (left, upper, right, lower) in enumerate(quadrants):
                    quadrant = img.crop((left, upper, right, lower)).resize((224, 224))
                    quadrant.save(os.path.join(splice_test_folder, f"{os.path.splitext(filename)[0]}_quad{i + 1}.png"))
        except Exception as e:
            print(f"Error processing {filename}: {e}")

def load_images_from_folder(folder_path, img_height, img_width, rescale):
    images, filenames = [], []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        try:
            img = Image.open(img_path).resize((img_width, img_height))
            images.append(np.array(img) * rescale)
            filenames.append(filename)
        except:
            continue
    return np.array(images), filenames

def calculate_cloud_coverage(image, t1=0.1, t2=0.1, kernel_size=10):
    kernel = np.ones((kernel_size, kernel_size), dtype=np.uint8)
    image = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
    
    R, G, B = image[:, :, 2].astype(float), image[:, :, 1].astype(float), image[:, :, 0].astype(float)
    
    rg = np.divide(R, G + 1e-10, out=np.zeros_like(R), where=G!=0)
    gb = np.divide(G, B + 1e-10, out=np.zeros_like(G), where=B!=0)
    
    cloud_mask = np.logical_and(np.abs(rg - 1) < t1, np.abs(gb - 1) < t2)
    cloud_mask = cv2.dilate(cloud_mask.astype(np.uint8), kernel, iterations=1)
    
    return np.sum(cloud_mask) / cloud_mask.size * 100

def save_vegetation_images(images, filenames, predicted_classes, save_folder):
    os.makedirs(save_folder, exist_ok=True)
    for img, filename, pred in zip(images, filenames, predicted_classes):
        if calculate_cloud_coverage(img) < 30 and pred == 1:
            Image.fromarray((img * 255).astype(np.uint8)).save(os.path.join(save_folder, filename))

def predict_with_tflite_model(interpreter, images):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    predictions = []
    for img in images:
        interpreter.set_tensor(input_details[0]['index'], np.expand_dims(img, axis=0).astype(np.float32))
        interpreter.invoke()
        predictions.append(interpreter.get_tensor(output_details[0]['index']))
    return np.array(predictions).flatten()

if __name__ == "__main__":
    start_time = time.time()

    splice_images(INPUT_FOLDER, SPLICE_TEST_FOLDER)

    interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
    interpreter.allocate_tensors()

    input_shape = interpreter.get_input_details()[0]['shape']
    img_height, img_width = input_shape[1], input_shape[2]
    rescale = 1.0 / 255

    test_images, test_filenames = load_images_from_folder(SPLICE_TEST_FOLDER, img_height, img_width, rescale)

    predictions = predict_with_tflite_model(interpreter, test_images)
    predicted_classes = np.round(predictions).astype(int)

    save_vegetation_images(test_images, test_filenames, predicted_classes, FILTERED_IMAGE_FOLDER)

    end_time = time.time()
    print(f"Total execution time: {end_time - start_time:.2f} seconds")


Total execution time: 11.58 seconds
